In [1]:
import os
import numpy as np
import keras
from keras.applications import ResNet50V2
from keras import models
from keras import layers
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras import optimizers
import matplotlib.pyplot as plt
import cv2
from keras.layers import Dropout
from tensorflow.keras import backend as K
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
import csv
import pandas as pd

Using TensorFlow backend.
C:\Users\idong\Anaconda3\envs\tens_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\idong\Anaconda3\envs\tens_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\idong\Anaconda3\envs\tens_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\idong\Anaconda3\envs\tens_gpu

# Data_Preprocessing

In [2]:
base_dir = r'D:\dataset\dental\data'

data = np.load(base_dir + '\\data_KNU_Diorco_so_imgs.npy')
label_csv= open(base_dir + '\\data_KNU_Diorco_so_img_labels.csv', 'r')
label = csv.reader(label_csv)

In [3]:
right_label = np.load(base_dir + '\\right_label.npy')
left_label = np.load(base_dir + '\\left_label.npy')

right = data[:,:,:, 0:3]
left = data[:,:,:, 3:7]

In [4]:
patient_datas = np.load(r'D:\dataset\dental\data\preprocessing\patient_data_2.npy', allow_pickle=True)
patient_labels = np.load(r'D:\dataset\dental\data\preprocessing\patient_label_2.npy', allow_pickle=True)

In [5]:
train_data, test_data, train_label, test_label= train_test_split(patient_datas, patient_labels, test_size = 0.2, random_state = 123)
train_data, val_data, train_label, val_label = train_test_split(train_data, train_label, test_size = 0.2, random_state = 123)

In [6]:
train_label = np.array(train_label)
test_label = np.array(test_label)
val_label = np.array(val_label)

In [7]:
#원 핫 인코딩을 위해 label을 바꿈
one_hot_train_labels = to_categorical(train_label - 1)
one_hot_validation_labels = to_categorical(val_label - 1)
one_hot_test_labels = to_categorical(test_label - 1)

In [8]:
real_train = []
real_val = []
real_test = []

def flat_datas(data, data_name):
    for m in range(len(data)):
        for j in range(4):
            data_name.append(data[m][j])

flat_datas(train_data, real_train)
flat_datas(val_data, real_val)
flat_datas(test_data, real_test)

In [9]:
train_data = np.array(real_train)
val_data = np.array(real_val)
test_data = np.array(real_test)

In [10]:
real_train_l = []
real_val_l = []
real_test_l = []

def flat_labels(label, label_name):
    for m in range(len(label)):
        for j in range(4):
            label_name.append(label[m][j])

flat_labels(one_hot_train_labels, real_train_l)
flat_labels(one_hot_validation_labels, real_val_l)
flat_labels(one_hot_test_labels, real_test_l)

In [11]:
one_hot_train_labels = np.array(real_train_l)
one_hot_val_labels = np.array(real_val_l)
one_hot_test_labels = np.array(real_test_l)

In [12]:
train_data = train_data.astype(np.float32) / 127.5 - 1
test_data = test_data.astype(np.float32) / 127.5 - 1
val_data = val_data.astype(np.float32) / 127.5 - 1

In [48]:
datagen = ImageDataGenerator(rotation_range = 10,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.1,
                                  zoom_range=0.1,
                                  horizontal_flip=False,
                                  vertical_flip=False,
                                  fill_mode='nearest'
                                 )

In [49]:
datagen = ImageDataGenerator(zca_whitening=True)

C:\Users\idong\Anaconda3\envs\tens_gpu\lib\site-packages\keras_preprocessing\image\image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


# Callback_clf

In [13]:
callbacks_list_clf = [
    keras.callbacks.EarlyStopping(
        monitor='acc',
        patience = 19,
        mode = max
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='real5_dental_ResNet50V2_clf.h5',
        monitor='val_acc',
        save_best_only=False,
        period = 1
    ),
    keras.callbacks.ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.1,
    patience = 10)
]

C:\Users\idong\Anaconda3\envs\tens_gpu\lib\site-packages\keras\callbacks\callbacks.py:788: RuntimeWarning: EarlyStopping mode <built-in function max> is unknown, fallback to auto mode.
  RuntimeWarning)


In [28]:
inp = layers.Input(shape=(480, 784, 3))
base_model = ResNet50V2(weights = 'imagenet',
                        input_tensor=inp,
                 include_top = False,
                 input_shape = (480, 784, 3))

In [29]:
#conv_base동결
base_model.trainable = False

In [30]:
len(base_model.trainable_weights)

0

In [22]:
base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 480, 784, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 486, 790, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 240, 392, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 242, 394, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [31]:
flat = layers.Flatten()
dropped = layers.Dropout(0.5)
global_average_layer = layers.GlobalAveragePooling2D()

dense1 = layers.Dense(256, activation='relu', name='dense_1')
prediction = layers.Dense(3, activation='softmax', name='prediction')

In [25]:
post_relu

In [39]:
post_relu = base_model.get_layer('post_relu')
x = global_average_layer(post_relu.output)
x = dense1(x)
x = dropped(x)
x = prediction(x)

model = models.Model(inputs = inp, outputs = x)

In [40]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 480, 784, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 486, 790, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 240, 392, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 242, 394, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

In [41]:
model.trainable = True

set_trainable = False
for layer in model.layers:
    if layer.name == 'global_average_pooling2d_2':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
        print(layer)
    else:
        layer.trainable = False

In [46]:
for layer in model.layers:
    if layer.trainable:
        print(layer.name)

global_average_pooling2d_2
dense_1
dropout_2
prediction


In [43]:
model.compile(optimizer=optimizers.Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])

In [50]:
history2 = model.fit_generator(
    datagen.flow(train_data, one_hot_train_labels, batch_size = 40),
    steps_per_epoch=45,
    epochs = 50,
    callbacks= callbacks_list_clf,
    validation_data=(val_data, one_hot_val_labels)
)

C:\Users\idong\Anaconda3\envs\tens_gpu\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\idong\Anaconda3\envs\tens_gpu\lib\site-packages\keras_preprocessing\image\image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/50
45/45 [==============================] - 93s 2s/step - loss: 1.0892 - acc: 0.4552 - val_loss: 1.2657 - val_acc: 0.3885
Epoch 2/50
45/45 [==============================] - 77s 2s/step - loss: 1.0214 - acc: 0.4916 - val_loss: 1.1608 - val_acc: 0.3885
Epoch 3/50
45/45 [==============================] - 76s 2s/step - loss: 0.9757 - acc: 0.5409 - val_loss: 1.1560 - val_acc: 0.3885
Epoch 4/50
45/45 [==============================] - 77s 2s/step - loss: 0.9369 - acc: 0.5633 - val_loss: 1.2310 - val_acc: 0.3885
Epoch 5/50
45/45 [==============================] - 77s 2s/step - loss: 0.9090 - acc: 0.5796 - val_loss: 1.2088 - val_acc: 0.3885
Epoch 6/50
45/45 [==============================] - 77s 2s/step - loss: 0.8680 - acc: 0.6122 - val_loss: 1.1979 - val_acc: 0.3885
Epoch 7/50
45/45 [==============================] - 77s 2s/step - loss: 0.8564 - acc: 0.6205 - val_loss: 1.2502 - val_acc: 0.4000
Epoch 8/50
45/45 [==============================] - 77s 2s/step - loss: 0.8319 - acc: 0.64

KeyboardInterrupt: 